### Plotting binned generated sample prices. 

The plot here assumes that you have two external processes running:

 * python generator.py     -i A.N.Other-Corp,0.3,123,100.0,0.01     -l 6000 -f 1s     -kb localhost:9092 -kt prices     -rt
 * python binning.py

In [1]:
import datetime
from pandas import Timestamp

from ipywidgets import interact

from bokeh.models.sources import ColumnDataSource
from bokeh.plotting import figure
from bokeh.io import push_notebook, show, output_notebook

from confluent_kafka import Consumer, KafkaError

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
consumer = Consumer({'bootstrap.servers': 'localhost:9092', 'group.id': 'test-group',
              'default.topic.config': {'auto.offset.reset': 'earliest'}})

consumer.subscribe(['prices', 'bin-prices'])

In [4]:
price_figure = figure(title='A.N.Other-Corp', plot_height=300, 
                      plot_width=600, y_range=(90, 110), x_axis_type='datetime')
price_figure.xaxis.axis_label = 'Time'
price_figure.yaxis.axis_label = 'Price'

price_data = ColumnDataSource(data=dict(x=[datetime.datetime(2017,1,1)], y=[100]))
price_line = price_figure.line(x="x", y="y", color="blue", source=price_data, legend='Price')

bin_data = ColumnDataSource(data=dict(x=[], y=[]))
bin_circle = price_figure.circle(x="x", y="y", color="red", source=bin_data, legend='Binned price')

handle = show(price_figure, notebook_handle=True)

xp, yp= [], []
updated_price_data = dict(x=xp, y=yp)
xb, yb= [], []
updated_bin_data = dict(x=xb, y=yb)

In [ ]:
def process_price(msg, x, y, updated_data, price_data):
    dt, sec, prc = msg.value().decode("utf-8").split(',')
    dt = Timestamp(dt).to_pydatetime()
    prc = float(prc)

    x.append(dt)
    y.append(prc)

    updated_data['x'] = x
    updated_data['y'] = y
    price_data.stream(updated_data, len(x))
    
def process_bin(msg, x, y, updated_data, bin_data):
    prc = float(msg.value().decode("utf-8"))
    
    dt, sec = msg.key().decode("utf-8").split(',')
    
    x.append(Timestamp(dt).to_pydatetime())
    y.append(prc)
    
    updated_data['x'] = x
    updated_data['y'] = y
    bin_data.stream(updated_data, len(x))

last_date = None
running = True
while running:
    msg = consumer.poll()
    if not msg.error():
        #print(f'Received message: {msg.value().decode("utf-8")}')
        if msg.topic() == 'prices':
             process_price(msg, xp, yp, updated_price_data, price_data)
        elif msg.topic() == 'bin-prices':
            process_bin(msg, xb, yb, updated_bin_data, bin_data)
        
        push_notebook(handle=handle)
    elif msg.error().code() != KafkaError._PARTITION_EOF:
        print(msg.error())
        running = False
        
c.close()